In [1]:
import sys
sys.path.append("/Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/geomeppy/_scripts")

from copy import deepcopy

from case_edits.ezcase import EzCase

from recipes.exp_01 import input, ns_axis

from geometry.zone import Zone
from geometry.wall import Wall

import pickle

import os

In [2]:
test_input = deepcopy(input)
test_ez = EzCase(ns_axis(input), RUN_CASE=False)

tests/test17_exp01
Added 4 walls 
Added 6 walls 
Added 4 walls 
Added 4 walls 
Added 6 walls 
Added 4 walls 
out.idf exists
IDF has changed: True
looking for sql


In [3]:
test_ez.prepare_plotter()

looking for sql


In [4]:
test_ez.run_analysis()

running analysis


In [5]:
test_ez.post_process_variables()

In [6]:
with open(os.path.join(test_ez.case.path, "post_process.pkl"), 'rb') as handle:
    b = pickle.load(handle)

In [7]:
for data in b['Zone Ventilation Net Heat Loss']:
    print(data.header.metadata)

{'type': 'Zone Ventilation Net Heat Loss', 'System': 'BLOCK 01 STOREY 0'}
{'type': 'Zone Ventilation Net Heat Loss', 'System': 'BLOCK 02 STOREY 0'}
{'type': 'Zone Ventilation Net Heat Loss', 'System': 'BLOCK 03 STOREY 0'}
{'type': 'Zone Ventilation Net Heat Loss', 'System': 'BLOCK 04 STOREY 0'}
{'type': 'Zone Ventilation Net Heat Loss', 'System': 'BLOCK 05 STOREY 0'}
{'type': 'Zone Ventilation Net Heat Loss', 'System': 'BLOCK 06 STOREY 0'}


In [ ]:
b

In [4]:
test_ez.case.idf_path

'cases/tests/test17_exp01/out.idf'

In [11]:
test_ez.afn.no_vent_zones

[Zone(Block 06), Zone(Block 03), Zone(Block 02)]

In [18]:
no_vent_walls = []

for zone in test_ez.afn.no_vent_zones:
    used_directions = []
    for w in zone.walls.values():
        assert type(w) == Wall
        if not w.partner_wall_name:
            print((w.display_name, w.partner_wall_name))
            used_directions.append(w.direction)
            no_vent_walls.append(w)

        if len(used_directions) >= 2:
            break

('Block 06 - West - W03', None)
('Block 06 - South - W04', None)
('Block 03 - South - W04', None)
('Block 02 - East - W01', None)
('Block 02 - North - W02', None)


In [17]:
no_vent_walls

[Wall(Block 06 - West - W03),
 Wall(Block 06 - South - W04),
 Wall(Block 03 - South - W04),
 Wall(Block 03 - South - W04),
 Wall(Block 03 - East - W01_1),
 Wall(Block 03 - North - W02_1),
 Wall(Block 03 - West - W03_1),
 Wall(Block 02 - East - W01),
 Wall(Block 02 - North - W02)]

In [21]:
min_flow_rate = test_ez.case.idf.newidfobject(
                "AirflowNetwork:MultiZone:SpecifiedFlowRate".upper()
            )
min_flow_rate.Air_Flow_Value


AIRFLOWNETWORK:MULTIZONE:SPECIFIEDFLOWRATE,
    ,                         !- Name
    ,                         !- Air Flow Value
    MassFlow;                 !- Air Flow Units

In [8]:
test_ez.case.geometry.walls["B_01_East_W01_1"].boundary_condition

In [ ]:
objs = test_ez.case.idf.idfobjects["AIRFLOWNETWORK:MULTIZONE:ZONE"]
objs

[
AIRFLOWNETWORK:MULTIZONE:ZONE,
    Block 01 Storey 0,        !- Zone Name
    Constant,                 !- Ventilation Control Mode
    ,                         !- Ventilation Control Zone Temperature Setpoint Schedule Name
    0,                        !- Minimum Venting Open Factor
    0,                        !- Indoor and Outdoor Temperature Difference Lower Limit For Maximum Venting Open Factor
    100,                      !- Indoor and Outdoor Temperature Difference Upper Limit for Minimum Venting Open Factor
    0,                        !- Indoor and Outdoor Enthalpy Difference Lower Limit For Maximum Venting Open Factor
    300000,                   !- Indoor and Outdoor Enthalpy Difference Upper Limit for Minimum Venting Open Factor
    ,                         !- Venting Availability Schedule Name
    Standard,                 !- Single Sided Wind Pressure Coefficient Algorithm
    10;                       !- Facade Width
, 
AIRFLOWNETWORK:MULTIZONE:ZONE,
    Block 02

In [ ]:
# need to find zones that have subsurfaces
zones_w_subsurfaces = []
other_zones = []
for s in test_ez.case.geometry.subsurfaces.values():
    zones_w_subsurfaces.append(s.wall.zone)

zones_w_subsurfaces = set(zones_w_subsurfaces)
zones_w_subsurfaces


{Zone(Block 01), Zone(Block 04), Zone(Block 05)}

In [ ]:
diff = set(test_ez.case.geometry.zone_list).difference(set(zones_w_subsurfaces))
for d in diff:
    print(d.name)

Block 06 Storey 0
Block 02 Storey 0
Block 03 Storey 0


In [ ]:
objs = test_ez.case.idf.idfobjects["AIRFLOWNETWORK:MULTIZONE:ZONE"]

for o in objs:
    if o.Zone_Name == list(diff)[0].name:
        print(o)


AIRFLOWNETWORK:MULTIZONE:ZONE,
    Block 06 Storey 0,        !- Zone Name
    Constant,                 !- Ventilation Control Mode
    ,                         !- Ventilation Control Zone Temperature Setpoint Schedule Name
    0,                        !- Minimum Venting Open Factor
    0,                        !- Indoor and Outdoor Temperature Difference Lower Limit For Maximum Venting Open Factor
    100,                      !- Indoor and Outdoor Temperature Difference Upper Limit for Minimum Venting Open Factor
    0,                        !- Indoor and Outdoor Enthalpy Difference Lower Limit For Maximum Venting Open Factor
    300000,                   !- Indoor and Outdoor Enthalpy Difference Upper Limit for Minimum Venting Open Factor
    ,                         !- Venting Availability Schedule Name
    Standard,                 !- Single Sided Wind Pressure Coefficient Algorithm
    10;                       !- Facade Width



In [ ]:
test_ez.afn.ep